In [404]:
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from datetime import date
import os

In [405]:
DARK_BLUE = '#3557A9'
DARK_GREEN = '#50974F'
DARK_RED = '#D43F2C'
DARK_YELLOW = '#F8D65A'

BLUE = '#627EC3'
GREEN = '#7ABA79'
RED = '#E36656'
YELLOW = '#FFE480'

LIGHT_BLUE = '#8CA5E2'
LIGHT_GREEN = '#A3D9A3'
LIGHT_RED = '#F6887A'
LIGHT_YELLOW = '#FFEA9D'

### Dados de acesso

In [406]:
import pandas as pd
# Caminho para o arquivo CSV
acesso_path = 'C:\\Users\\leibr\\Documents\\Lappis\\conjuv\\data\\conjuv_11-09.csv'

# Carregar o arquivo topics.csv em UTF-8 e usar ponto e vírgula como separador
acesso_path = pd.read_csv(acesso_path, sep=';', encoding='utf-8')

selected_columns = ['Visualizações únicas de páginas', 'Visualizações de páginas', 'Tempo médio na página', 'Taxa de ressalto']

# Selecionar as colunas desejadas
acesso_tabela = acesso_path[selected_columns]

acesso_tabela = tabulate(acesso_tabela, headers='keys', tablefmt='pipe')

In [407]:
proposals_path = 'C:\\Users\\leibr\\Documents\\Lappis\\conjuv\\data\\conjuv_propostas.csv'

df_sheets = pd.read_csv(proposals_path, sep=';', encoding='utf-8')

import pandas as pd


data = (df_sheets['participatory_space/url'] == 'http://brasilparticipativo.presidencia.gov.br/assemblies/confjuv4?assembly_slug=confjuv4') & \
         (~df_sheets['state'].isin(['rejected', 'withdrawn']))


new_data = df_sheets[data]

report_path = 'relatorio_conjuv.md'

### Top 20 propostas por voto

In [408]:

new_data = new_data.sort_values(by='supports', ascending=False)

top_20_programas = new_data.head(20)

tabela_ranking = top_20_programas[['id', 'title/pt-BR', 'category/name/pt-BR', 'supports', 'comments']].copy()
tabela_ranking.columns = ['ID', 'Propostas', 'Temas', 'Nº de Votos', 'Nº de Comentários']


# URL base para as propostas
base_url = "http://brasilparticipativo.presidencia.gov.br/assemblies/confjuv4/f/10/proposals/"

# Função para criar o link para cada proposta
def create_link(proposal_id, proposal_name):
    url = f"{base_url}{proposal_id}"
    return f"[{proposal_name}]({url})"

# Aplicar a função para criar os links na coluna 'Propostas'
tabela_ranking['Propostas'] = tabela_ranking.apply(lambda row: create_link(row['ID'], row['Propostas']), axis=1)

# Formatar a tabela em Markdown
tabela_ranking_markdown = tabulate(tabela_ranking, headers='keys', tablefmt='pipe', showindex=False)


### Ranking por tema

In [409]:

temas_df = new_data.groupby('category/name/pt-BR').agg({
    'title/pt-BR': 'nunique',  
    'supports': 'sum',        
    'comments': 'sum'        
}).reset_index()

temas_df.columns = ['Temas', 'Nº de Propostas', 'Nº de Votos', 'Nº de Comentários']

temas_df_markdown = tabulate(temas_df, headers='keys', tablefmt='pipe')

### Ranking Tópicos

In [410]:
# Caminho para o arquivo CSV
topics_path = 'C:\\Users\\leibr\\Documents\\Lappis\\conjuv\\data\\topics.csv'

# Carregar o arquivo topics.csv em UTF-8 e usar ponto e vírgula como separador
topics_df = pd.read_csv(topics_path, sep=';', encoding='utf-8')

# Selecionar as colunas desejadas
selected_columns = ['tema', 'titulo', 'proposta', 'Tópico 1', 'Tópico 2', 'Tópico 3', 'Tópico 4', 'Tópico 5']
topics_df = topics_df[selected_columns]

# Função para extrair o nome do tópico sem a porcentagem
def extract_topic_name(topic_with_percentage):
    if isinstance(topic_with_percentage, str):
        return topic_with_percentage.split('(')[0].strip()
    else:
        return ''

# Aplicar a função para remover a porcentagem dos tópicos
for col in ['Tópico 1', 'Tópico 2', 'Tópico 3', 'Tópico 4', 'Tópico 5']:
    topics_df[col] = topics_df[col].apply(extract_topic_name)


In [411]:
import pandas as pd

# Suponhamos que você já carregou e pré-processou os dados conforme mencionado anteriormente.

# Função para contar propostas por tópico em uma coluna
def count_proposals_by_topic(topic_column):
    topics = topic_column.str.split(', ')
    flattened_topics = [item.strip() for sublist in topics.dropna() for item in sublist]
    topic_counts = pd.Series(flattened_topics).value_counts()
    return topic_counts

# Contagem de propostas por tópico nas colunas 'Tópico 1' e 'Tópico 2'
topic_1_counts = count_proposals_by_topic(topics_df['Tópico 1'])
topic_2_counts = count_proposals_by_topic(topics_df['Tópico 2'])

# Crie DataFrames para os top 10 tópicos em 'Tópico 1' e 'Tópico 2'
top_10_topic_1 = pd.DataFrame({'Tópico 1': topic_1_counts.index[:10], 'Qtd Propostas': topic_1_counts.values[:10]})
top_10_topic_2 = pd.DataFrame({'Tópico 2': topic_2_counts.index[:10], 'Qtd Propostas': topic_2_counts.values[:10]})

# Combine os DataFrames
combined_df = pd.concat([top_10_topic_1, top_10_topic_2], axis=1)


In [412]:
total_propostas = len(new_data)
propostas_acima_de_30_votos = len(new_data[new_data['supports'] > 30])
propostas_abaixo_de_10_votos = len(new_data[new_data['supports'] < 10])
porcentagem_abaixo_de_10_votos = (propostas_abaixo_de_10_votos / total_propostas) * 100
menor_votos_entre_propostas = new_data['supports'].min()

In [413]:

titulo = f"<center>Relatório Conjuv ({date.today().strftime('%d/%m/%Y')})</center>"

report_path = 'relatorio_conjuv.md'


with open(report_path, 'w', encoding='utf-8') as report_file:
    report_file.write(f"# {titulo}\n\n")
    report_file.write("</p>\n\n")

with open(report_path, 'a', encoding='utf-8') as report_file:
    report_file.write("\n# Sumário:\n")
    report_file.write("- [Dados de Acesso](#dados-de-acesso)\n")
    report_file.write("- [TOP 20 Propostas por Voto no Geral](#top-20-propostas-por-voto-no-geral)\n")
    report_file.write("- [Análise Qualitativa das Propostas](#análise-qualitativa-das-propostas)\n")
    report_file.write("- [Ranking por Tópicos](#ranking-por-tópicos)\n")

with open(report_path, 'a', encoding='utf-8') as report_file:
    report_file.write(f'## *Dados de Acesso*\n{acesso_tabela}\n\n')
    report_file.write(f'### **Perfil do visitante**\n\n')
    report_file.write('\n<p align="center">\n')
    report_file.write('  <img src="../conjuv/data/painel_visitante.png" alt="Painel de um visitante"/>\n')
    report_file.write('</p>\n\n')

    report_file.write(f"## *TOP 20 Propostas por Voto no Geral*\n{tabela_ranking_markdown}\n\n")
    report_file.write(f"## *Análise Qualitativa das Propostas* \n Os temas com o maior número de propostas são: \n{temas_df_markdown}\n")
    report_file.write(f"- De {total_propostas} propostas atuais, apenas {propostas_acima_de_30_votos} delas apresentam votação acima de 30;\n")
    report_file.write(f"- {propostas_abaixo_de_10_votos} propostas estão com menos de 10 votos ({porcentagem_abaixo_de_10_votos:.2f}% do total de propostas);\n")
    report_file.write(f"- Todos os programas apresentam votação acima de {menor_votos_entre_propostas};\n")
    report_file.write(f"## *Ranking por Tópicos*\n")
    report_file.write(tabulate(combined_df, headers='keys', tablefmt='pipe', showindex=False))

   

